In [1]:
import pandas as pd
from pymongo import MongoClient
from pymongo.database import Database

database = MongoClient('mongodb://172.30.1.20:50102')

database['admin'].authenticate('worker', 'worker@sofie')

coll_card = database['config']['card']

In [4]:
QUERY = {'company': {'$in': ['abril', 'caras']}, '__archived__': {'$exists': False}}

PROJECTION = {'_id': 0,'company': 1, 'name': 1, 'title': 1, 'payment_conditions': 1, 'bookmarks': 1, 'reference_codes': 1}

COLUMNS = ['LINK', 'EMPRESA', 'CARD', 'TÍTULO', 'MODALIDADE', 'EDIÇÕES', 'PERIODICIDADE', 'PREÇO SOFIE', 'PARCELAS', 'VALOR PARCELA', 'PRÊMIO', 'TAXA', 'REPASSE', 'REFERÊNCIA 1', 'REFERÊNCIA 2']
RESULT = list()

def higieniza(value):
    return '{:.2f}'.format(value).replace('.', ',')

for item in coll_card.find(QUERY, PROJECTION):
    LAST_CONDITION = item['payment_conditions'][-1]
    BOOK_CARD = item['bookmarks']['card']
    
    RESULT.append({
        'LINK': f'https://jmarioguedes.mysofie.com/offer/{item["name"]}',
        'EMPRESA': item['company'],
        'CARD': item['name'],
        'TÍTULO': item['title']['title'],
        'MODALIDADE': item['title']['sub_title'],
        'EDIÇÕES': f"{BOOK_CARD['row_1']} {BOOK_CARD['row_2']} - {BOOK_CARD['row_3']}",
        'PERIODICIDADE':item['bookmarks']['detail_1']['row_2'],
        'PREÇO SOFIE': higieniza(LAST_CONDITION['price']),
        'PARCELAS': LAST_CONDITION['quotes']['qtt'],
        'VALOR PARCELA': higieniza(LAST_CONDITION['quotes']['value']),
        'PRÊMIO': higieniza(LAST_CONDITION['reward']),
        'TAXA': higieniza(LAST_CONDITION['fee']),
        'REPASSE': higieniza(LAST_CONDITION['price'] - LAST_CONDITION['reward'] - LAST_CONDITION['fee']),
        'REFERÊNCIA 1': item['reference_codes']['reference_1'],
        'REFERÊNCIA 2': item['reference_codes']['reference_2']
    })

In [5]:
planilha = pd.DataFrame(RESULT, columns=COLUMNS)
planilha

,LINK,EMPRESA,CARD,TÍTULO,MODALIDADE,EDIÇÕES,PERIODICIDADE,PREÇO SOFIE,PARCELAS,VALOR PARCELA,PRÊMIO,TAXA,REPASSE,REFERÊNCIA 1,REFERÊNCIA 2
0,https://jmarioguedes.mysofie.com/offer/498629a...,caras,498629a9-35fb-4ac1-b4c8-f2f070d989dd,Revista Caras,Impressa,52 EDIÇÕES - ANUAL,AVULSO,"530,40",8,"66,30","97,00","0,01","433,39",1150,52676
1,https://jmarioguedes.mysofie.com/offer/3dadce9...,caras,3dadce97-800b-4a9d-9c59-e84c61c9bbf9,Revista Caras,Impressa,104 EDIÇÕES - BIANUAL,AVULSO,"1060,80",12,"88,40","180,00","0,01","880,79",1150,52677
2,https://jmarioguedes.mysofie.com/offer/db26e95...,caras,db26e95f-0213-4203-9195-81b7570c6451,Aventuras na História,Impressa,12 EDIÇÕES - ANUAL,AVULSO,"180,00",2,"90,00","36,00","0,01","143,99",1148,50689
3,https://jmarioguedes.mysofie.com/offer/d0a2057...,caras,d0a2057e-a092-41ae-949a-356a859828f2,Aventuras na História,Impressa,24 EDIÇÕES - BIANUAL,AVULSO,"360,00",4,"90,00","70,00","0,01","289,99",1148,51386
4,https://jmarioguedes.mysofie.com/offer/020dd17...,caras,020dd17c-e5e4-4851-b91f-9f9881a0ae7b,Contigo!,Impressa,12 EDIÇÕES - ANUAL,AVULSO,"132,00",2,"66,00","26,00","0,01","105,99",1154,50685
5,https://jmarioguedes.mysofie.com/offer/893b062...,caras,893b0627-cc99-43b4-ad4d-7e1414f9e25b,Contigo!,Impressa,24 EDIÇÕES - BIANUAL,AVULSO,"264,00",4,"66,00","51,00","0,01","212,99",1154,50686
6,https://jmarioguedes.mysofie.com/offer/0edf831...,abril,0edf831c-abfa-49c4-ad0e-737b99b5d9c1,Claudia,Digital,12 EDIÇÕES - ANUAL,MENSAL,"86,40",4,"21,60","13,29","11,55","61,56",,
7,https://jmarioguedes.mysofie.com/offer/e47b833...,abril,e47b8333-e156-47fd-bbc0-12246a6482a4,Claudia,Digital,24 EDIÇÕES - BIANUAL,MENSAL,"172,80",4,"43,20","26,57","23,11","123,12",,
8,https://jmarioguedes.mysofie.com/offer/bb2f609...,abril,bb2f6098-0536-4e05-aa48-d987f92dbb27,Combo Claudia,Digital + Impressa,24 EDIÇÕES - BIANUAL,MENSAL,"239,33",12,"19,94","31,86","37,84","169,63",,
9,https://jmarioguedes.mysofie.com/offer/06e8e2c...,abril,06e8e2c2-b7d8-4098-84ea-05c87ffcba66,Combo Dossiê Superinteressante,Digital + Impressa,24 EDIÇÕES - BIANUAL,MENSAL,"233,28",12,"19,44","31,06","36,88","165,34",,


# GERANDO CSV

In [6]:
import csv

with open('c:/teste/saida.csv', 'w', newline='') as arquivo:
    writer = csv.DictWriter(arquivo, fieldnames=COLUMNS, delimiter=';')
    writer.writeheader()
    for item in RESULT:
        writer.writerow(item)

# ACERTANDO OS CÓDIGOS DE REFERÊNCIA

In [7]:
LISTAGEM = """
0edf831c-abfa-49c4-ad0e-737b99b5d9c1|1010|12
e47b8333-e156-47fd-bbc0-12246a6482a4|1010|24
bb2f6098-0536-4e05-aa48-d987f92dbb27|3435|24
06e8e2c2-b7d8-4098-84ea-05c87ffcba66|3439|24
16e771ad-1e7c-4ca9-9b6c-77cce3e9d8fa|893|26
6645bcf8-cae8-4aa1-8dcb-9d3f9d2d35e6|893|52
2533cf0e-e01a-4517-ac9e-46c528566346|1019|12
51eb372f-7ac4-4d17-8620-00813c1c08e2|1019|24
752d8b6b-7c4c-4db7-b766-68d95633c13a|3451|24
b7ffe08b-f488-4c0d-9fd3-5e7d7d211f1f|1014|14
97a2d0a4-17fa-444f-96b2-c4f43ecbf4da|1014|28
6119aeae-c19a-4b3a-8cac-d3c26f9aa63d|3453|28
e6c30bec-0b61-43c4-87de-8298752a4874|3862|26
da6ada2b-d239-4ac4-9959-a8010514449b|1009|14
e08bec69-8ae1-4bdb-a647-ac890124c917|1009|28
8721be44-b631-43e7-8085-1aa1a01cfc8f|3457|28
8bc50e9d-a76d-4bbe-a50a-9365e23aec98|883|52
7f36bfca-d856-4856-bf70-4e97a0c2aefc|883|104
9120fb8e-2c66-4ccd-b50f-bff3dd942a8f|3425|104
0bf66fdc-375f-437b-a4e8-6056435e2578|3463|12
43772d87-4c2f-4c84-9baf-fc97448c57e3|1021|12
201db940-abde-4bab-8691-715d160e1f31|1021|24
c6fe413f-cb57-4299-b2d4-d0c930c7af3f|3465|24
"""

In [9]:
for item in LISTAGEM.split():
    card, ref1, ref2 = item.split('|')
    coll_card.update_one({"name": card}, {'$set': {'reference_codes': {'reference_1': ref1, 'reference_2': ref2}}})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until
